In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
df_green = spark.read.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/data/pq/green/*/*')

In [4]:
df_green.registerTempTable('green')

/usr/local/Cellar/apache-spark/3.2.1/libexec/python/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [8]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999994|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [9]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/data/report/revenue/green', mode='overwrite')

In [10]:
df_yellow = spark.read.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/data/pq/yellow/*/*')

In [11]:
df_yellow.registerTempTable('yellow')

/usr/local/Cellar/apache-spark/3.2.1/libexec/python/pyspark/sql/dataframe.py:140: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  FutureWarning


In [12]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [13]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/data/report/revenue/yellow', mode='overwrite')

In [15]:
df_green_revenue_tmp = df_green_revenue \
                        .withColumnRenamed('amount', 'green_amount') \
                        .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
                        .withColumnRenamed('amount', 'yellow_amount') \
                        .withColumnRenamed('number_records', 'yellow_number_records')

In [16]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [18]:
df_join.write.parquet('/Users/amey/git/data-engineering-zoomcamp-amey/5_Week_5_batch_processing/data/report/revenue/total', mode='overwrite')

In [19]:
SparkSession.stop(spark)